In [20]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import pandas as pd
from nltk.stem import WordNetLemmatizer
from string import digits
from nltk.corpus import stopwords
import string
import nltk
from sklearn.metrics import confusion_matrix
from random import randint
import numpy as np
import torch
import sklearn
import operator
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Preprocessing the data


In [2]:
"""
Not removing digits as date-time may also lead to a pattern as per problem statement
By analysing the data, Key words like "Please" "Could" should not be removed as they are also leading to a pattern.
So here I am just removing stopwords which has length <=3
"""
lemmatizer = WordNetLemmatizer()
def preProcess(sentence):
    stop = set(stopwords.words('english'))
    remove_punctuation = str.maketrans('','',string.punctuation)
#     remove_digits = str.maketrans('','',digits)
    sentence = sentence.translate(remove_punctuation)
#     sentence = sentence.translate(remove_digits).translate(remove_punctuation)
    tokens = [i for i in sentence.lower().split() if (i not in stop or len(i) > 3)]
    s=""
    for token in tokens:
        token = lemmatizer.lemmatize(token)
        s = s + ' ' + token
    return s;

# Loading the data from train and test files

In [3]:
df = pd.read_csv("/home/varun/Downloads/train.csv")

In [4]:
df["NewText"] = ""
for i in range(df.shape[0]):
    df["NewText"][i] = preProcess(df["Text"][i])
df['Tag'] = df['Label'].eq('Yes').astype(int) 

In [5]:
df1 = pd.read_csv("/home/varun/Downloads/test.csv")
df1["NewText"] = ""
for i in range(df1.shape[0]):
    df1["NewText"][i] = preProcess(df1["Text"][i])
df1['Tag'] = df1['Label'].eq('Yes').astype(int)

In [6]:
list1  = df["NewText"].tolist()
list2  = df1["NewText"].tolist() 
for i in range(len(list2)):
    list1.append(list2[i])

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/varun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Loading Infersent Model

In [8]:
from models import InferSent
model_version = 1
MODEL_PATH = "../infersent%s.pickle" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [9]:
use_cuda = False
model = model.cuda() if use_cuda else model

In [10]:
W2V_PATH = '../dataset/GloVe/glove.840B.300d.txt' if model_version == 1 else '../dataset/fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)

In [11]:
model.build_vocab_k_words(K=100000)

Vocab size : 100000


In [12]:
X = model.encode(list1)

In [13]:
"""
Evey sentence is converted to 1*4096 vector

"""
X.shape

(4649, 4096)

In [14]:
"""
As we appended test data for vectorizing, 
we put last 992 files in the X_test and first 3657 files in the X_train
"""

X_train = X[:3657]
X_test = X[3657:]
y_train = df['Tag'].tolist()
y_test = df1['Tag'].tolist()
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
y_train = np.reshape(y_train,(3657,1))
y_test = np.reshape(y_test,(992,1))

In [15]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3657, 4096)
(992, 4096)
(3657, 1)
(992, 1)


# KNN Classifier
## accuracy = 57.4%

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [17]:
knn = KNeighborsClassifier(n_neighbors= 30)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy_score(y_test,y_pred)

/home/varun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


0.5745967741935484

In [18]:
confusion_matrix(y_test, y_pred)

array([[302, 381],
       [ 41, 268]])

In [21]:
print("Recall : ", precision_score(y_test, y_pred))
print("Precesion : ", recall_score(y_test, y_pred))

Recall :  0.41294298921417566
Precesion :  0.8673139158576052


# SVM Classifier
## accuracy = 75.3%

In [22]:
X_train = X[:3657]
X_test = X[3657:]
y_train = df['Tag'].tolist()
y_test = df1['Tag'].tolist()
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
y_train = np.reshape(y_train,(3657,1))
y_test = np.reshape(y_test,(992,1))

In [23]:
from sklearn.svm import SVC
clf = SVC(C=2300, kernel='rbf')
clf.fit(X_train, y_train)

/home/varun/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/varun/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=2300, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [24]:
y_pred = clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7530241935483871

In [25]:
confusion_matrix(y_test, y_pred)

array([[510, 173],
       [ 72, 237]])

In [26]:
print("Recall : ", precision_score(y_test, y_pred))
print("Precesion : ", recall_score(y_test, y_pred))

Recall :  0.5780487804878048
Precesion :  0.7669902912621359


# XGB Classifier
## accuracy = 76.2%

In [27]:
from xgboost import XGBClassifier

In [28]:
model = XGBClassifier()
model.fit(X_train, y_train)

/home/varun/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/varun/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [29]:
y_pred = model.predict(X_test)
accuracy_score(y_test,y_pred)

0.7620967741935484

In [30]:
confusion_matrix(y_test, y_pred)

array([[533, 150],
       [ 86, 223]])

In [31]:
print("Recall : ", precision_score(y_test, y_pred))
print("Precesion : ", recall_score(y_test, y_pred))

Recall :  0.5978552278820375
Precesion :  0.7216828478964401
